In [3]:
#sudo pip install ibm_db
import ibm_db
# conda install -c conda-forge ibm_db

import pandas as pd
import numpy as np
# import psycopg2
# conda install -c anaconda psycopg2
import sqlalchemy
# conda install -c anaconda sqlalchemy
import matplotlib as plt
import requests
# conda install -c anaconda requests
from requests.auth import HTTPDigestAuth
import json
# conda install -c jmcmurray json
import urllib.request
import urllib.parse
# conda install -c anaconda zlib
import sys
import asyncio

from sqlalchemy import create_engine

# import ibm_db


import ibm_db
import ibm_db_sa
# conda install -c auto ibm_db_sa
# pip install ibm-db-sa

import ibm_db_dbi

import logging

import datetime

from my_functions import *

In [4]:
logging.basicConfig(filename='logs.log',level=logging.DEBUG)


POSTGRES_ADDRESS = 'localhost'
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'ebreshna_user'
POSTGRES_PASSWORD = 'secret'
POSTGRES_DBNAME = 'ebreshna_test'
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME, password=POSTGRES_PASSWORD, ipaddress=POSTGRES_ADDRESS, port=POSTGRES_PORT, dbname=POSTGRES_DBNAME))
# conn = psycopg2.connect("host=localhost dbname=asims_test user=asims_user password=secret port=5432")
# cur = conn.cursor()
psql_engine = create_engine(postgres_str)

# SQL SERVER CONNECTION
DB2_ADDRESS = '10.10.100.47'
DB2_PORT = '53000'
DB2_USERNAME = 'db2inst1'
DB2_PASSWORD = 'db2inst1'
DB2_DBNAME = 'mpower'
# db2_engine = create_engine('ibm_db_sa+pyodbc:///?odbc_connect={}'.format(quoted))
# db2_engine = create_engine("db2+ibm_db://user:pass@10.10.100.47:51000/mpower")
# conn = ibm_db.connect("DATABASE=mpower;HOSTNAME=host;PORT=50000;PROTOCOL=TCPIP;UID=username;PWD=password;", "", "")
db2_conn = ibm_db.connect("DRIVER={{IBM DB2 ODBC DRIVER}};DATABASE=mpower;HOSTNAME=10.10.100.47;PORT=53000;PROTOCOL=TCPIP;UID=db2inst1;PWD=db2inst1;", "", "")
# this is required to enable database access through Pandas
db2_conni =  ibm_db_dbi.connect(u"DRIVER={{IBM DB2 ODBC DRIVER}};DATABASE=mpower;HOSTNAME=10.10.100.47;PORT=53000;PROTOCOL=TCPIP;UID=db2inst1;PWD=db2inst1;")

In [5]:
# Region
cols_map = [
    {'src_col': 'ID', 'des_col': 'id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CONS_NUMBER', 'des_col': 'cons_number', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PRST_RDG_DATE', 'des_col': 'prst_rdg_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'PRST_STATUS_KWH', 'des_col': 'prst_status_kwh', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PRST_STATUS_KVA', 'des_col': 'prst_status_kva', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PRST_STATUS_KVAH', 'des_col': 'prst_status_kvah', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PRST_RDG_KWH', 'des_col': 'prst_rdg_kwh', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'PRST_RDG_KVA', 'des_col': 'prst_rdg_kva', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PRST_RDG_KVAH', 'des_col': 'prst_rdg_kvah', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PREV_RDG_DATE', 'des_col': 'prev_rdg_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'PREV_STATUS_KWH', 'des_col': 'prev_status_kwh', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PREV_STATUS_KVA', 'des_col': 'prev_status_kva', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PREV_STATUS_KVAH', 'des_col': 'prev_status_kvah', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PREV_RDG_KWH', 'des_col': 'prev_rdg_kwh', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'PREV_RDG_KVA', 'des_col': 'prev_rdg_kva', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'PREV_RDG_KVAH', 'des_col': 'prev_rdg_kvah', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'MF_KWH', 'des_col': 'mf_kwh', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'MF_KVA', 'des_col': 'mf_kva', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'MF_KVAH', 'des_col': 'mf_kvah', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'DF_KWH', 'des_col': 'df_kwh', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'DF_KVA', 'des_col': 'df_kva', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'DF_KVAH', 'des_col': 'df_kvah', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'RECORDED_KWH', 'des_col': 'recorded_kwh', 'DECIMAL': 'String', 'des_dtype': 'numeric'},
    {'src_col': 'RECORDED_KVA', 'des_col': 'recorded_kva', 'DECIMAL': 'String', 'des_dtype': 'numeric'},
    {'src_col': 'RECORDED_KVAH', 'des_col': 'recorded_kvah', 'DECIMAL': 'String', 'des_dtype': 'numeric'},
    {'src_col': 'CREATED_BY', 'des_col': 'created_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CREATE_DATE', 'des_col': 'create_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'UPDATED_BY', 'des_col': 'updated_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'UPDATE_DATE', 'des_col': 'update_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'DB_IP_ADDRESS', 'des_col': 'db_ip_address', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SOURCE_OFFICE_ID', 'des_col': 'source_office_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'TARGET_OFFICE_ID', 'des_col': 'target_office_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'RECORD_STATUS', 'des_col': 'record_status', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SCREEN_ID', 'des_col': 'screen_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MONTH', 'des_col': 'month', 'src_dtype': 'DECIMAL', 'des_dtype': 'String'},
    {'src_col': 'YEAR', 'des_col': 'year', 'src_dtype': 'DECIMAL', 'des_dtype': 'String'},
    {'src_col': 'FLAG1', 'des_col': 'flag1', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG2', 'des_col': 'flag2', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG3', 'des_col': 'flag3', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG4', 'des_col': 'flag4', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG5', 'des_col': 'flag5', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MODULE_ID', 'des_col': 'module_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SUBMODULE_ID', 'des_col': 'submodule_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'COMM_MODE_ID', 'des_col': 'comm_mode_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MTR_RDR_ID', 'des_col': 'mtr_rdr_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MTR_CHG_DT', 'des_col': 'mtr_chg_dt', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'NORMAL_STATUS_DT', 'des_col': 'normal_status_dt', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'OBSERVATION_ID', 'des_col': 'observation_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'RDG_MONTH', 'des_col': 'rdg_month', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'RDG_YEAR', 'des_col': 'rdg_year', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'RECOMM_KWH', 'des_col': 'recomm_kwh', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'RECOMM_KVA', 'des_col': 'recomm_kva', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'RECOMM_KVAH', 'des_col': 'recomm_kvah', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'STATUS_DATE', 'des_col': 'status_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'RDG_USE', 'des_col': 'rdg_use', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'AVG_UNITS_KWH', 'des_col': 'avg_units_kwh', 'src_dtype': 'DECIMAL', 'des_dtype': 'String'},
    {'src_col': 'AVG_UNITS_KVA', 'des_col': 'avg_units_kva', 'src_dtype': 'DECIMAL', 'des_dtype': 'String'},
    {'src_col': 'AVG_UNITS_KVAH', 'des_col': 'avg_units_kvah', 'src_dtype': 'DECIMAL', 'des_dtype': 'String'},
    {'src_col': 'EMPLOYEE_ID', 'des_col': 'employee_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'REMARKS', 'des_col': 'remarks', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'BEHAVIOUR_KWH', 'des_col': 'behaviour_kwh', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'BEHAVIOUR_KVA', 'des_col': 'behaviour_kva', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'BEHAVIOUR_KVAH', 'des_col': 'behaviour_kvah', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'TOD_ID', 'des_col': 'tod_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PREV_TOD_VOLTAGE', 'des_col': 'prev_tod_voltage', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'PREV_TOD_CURRENT', 'des_col': 'prev_tod_current', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'PREV_TOD_PF', 'des_col': 'prev_tod_pf', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PRES_TOD_VOLTAGE', 'des_col': 'pres_tod_voltage', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'PRES_TOD_CURRENT', 'des_col': 'pres_tod_current', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'PRES_TOD_PF', 'des_col': 'pres_tod_pf', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'REC_TOD_VOLTAGE', 'des_col': 'rec_tod_voltage', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'REC_TOD_CURRENT', 'des_col': 'rec_tod_current', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'PREV_BD_KVA', 'des_col': 'prev_bd_kva', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'PRST_BD_KVA', 'des_col': 'prst_bd_kva', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'PROV_FLAG', 'des_col': 'prov_flag', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CHECK_CONDITION', 'des_col': 'check_condition', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'METERNO', 'des_col': 'meterno', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'BILL_REC_KWH', 'des_col': 'bill_rec_kwh', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'DELAYED_READING', 'des_col': 'delayed_reading', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'OFFICE_ID', 'des_col': 'office_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CURR_CAT', 'des_col': 'curr_cat', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CURR_PHASE', 'des_col': 'curr_phase', 'src_dtype': 'String', 'des_dtype': 'integer'},
    {'src_col': 'LOAD_TYPE', 'des_col': 'load_type', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ANOMALIES', 'des_col': 'anomalies', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NEW_MTRNO', 'des_col': 'new_mtrno', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PROVISIONAL', 'des_col': 'provisional', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CYCLE', 'des_col': 'cycle', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ROUTE_CODE', 'des_col': 'route_code', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ROUTE_CODE_DHARI', 'des_col': 'route_code_dhari', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ANOMALIES_PREV', 'des_col': 'anomalies_prev', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'OLDPRST_READING', 'des_col': 'oldprst_reading', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'OLDPRST_STATUS', 'des_col': 'oldprst_status', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'RDG_UPDATED_BY', 'des_col': 'rdg_updated_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DHARI_YEAR', 'des_col': 'dhari_year', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'LATITUDE', 'des_col': 'latitude', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'LONGITUDE', 'des_col': 'longitude', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'AVG_UNITS', 'des_col': 'avg_units', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'DEFAVG_UNITS', 'des_col': 'defavg_units', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'EXCP', 'des_col': 'excp', 'src_dtype': 'String', 'des_dtype': 'numeric'}
]

migrated_row_count = migrate_data(db2_conni, psql_engine, 'CENTRAL_ALL.LTMTR_RDG_III_T', 'public', 'ltmtr_rdg_iii_t', cols_map)
print("Total Migrated Records (ltmtr_rdg_iii_t): " + str(migrated_row_count))


Total Migrated Records (ltmtr_rdg_iii_t): Total Src Records: 737 Total migrated Records to dest: 739
